In [1]:
import math
import numpy as np
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy import stats
import pandas as pd
import geopandas as gpd
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.ensemble import IsolationForest
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import gaussian_kde, pearsonr
from matplotlib.ticker import FixedLocator

In [2]:
# Carregar shapefiles
gdf_model = gpd.read_file(r"../../Data/Processed/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_model.shp")
gdf_log = gpd.read_file(r"../../Data/Processed/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_model_log.shp")

# Converter para DataFrame puro (sem geometria)
df_model = pd.DataFrame(gdf_model.drop(columns='geometry'))
df_log = pd.DataFrame(gdf_log.drop(columns='geometry'))

c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


In [3]:
'''def remove_outliers_iqr_sync(*dfs, k=1.5):
    """
    Remove outliers de múltiplos dataframes de forma sincronizada usando IQR.
    
    Parâmetros:
        dfs: dataframes relacionados (mesmo número de linhas, diferentes transformações)
        k: fator IQR (default 1.5)
    
    Retorna:
        lista de dataframes limpos, com linhas sincronizadas
    """
    base_df = dfs[0]
    df_num = base_df.select_dtypes(include=np.number)
    Q1 = df_num.quantile(0.25)
    Q3 = df_num.quantile(0.75)
    IQR = Q3 - Q1
    
    # Máscara de linhas que não são outliers
    mask = ~((df_num < (Q1 - k*IQR)) | (df_num > (Q3 + k*IQR))).any(axis=1)
    
    # Aplica a mesma máscara a todos os dataframes
    return [df.loc[mask] for df in dfs]

# Uso
df_model, df_log = remove_outliers_iqr_sync(df_model, df_log)'''

'def remove_outliers_iqr_sync(*dfs, k=1.5):\n    """\n    Remove outliers de múltiplos dataframes de forma sincronizada usando IQR.\n\n    Parâmetros:\n        dfs: dataframes relacionados (mesmo número de linhas, diferentes transformações)\n        k: fator IQR (default 1.5)\n\n    Retorna:\n        lista de dataframes limpos, com linhas sincronizadas\n    """\n    base_df = dfs[0]\n    df_num = base_df.select_dtypes(include=np.number)\n    Q1 = df_num.quantile(0.25)\n    Q3 = df_num.quantile(0.75)\n    IQR = Q3 - Q1\n\n    # Máscara de linhas que não são outliers\n    mask = ~((df_num < (Q1 - k*IQR)) | (df_num > (Q3 + k*IQR))).any(axis=1)\n\n    # Aplica a mesma máscara a todos os dataframes\n    return [df.loc[mask] for df in dfs]\n\n# Uso\ndf_model, df_log = remove_outliers_iqr_sync(df_model, df_log)'

In [4]:
# Variáveis a plotar (todas exceto 'ros_p')
vars_to_plot = [col for col in df_model.columns if col != 'ros_p']

In [5]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import gaussian_kde
from sklearn.metrics import r2_score

def signed_log(x):
    """Apply signed log transformation: sign(x) * log(|x| + 1)"""
    return np.sign(x) * np.log(np.abs(x) + 1)

def signed_exp(x):
    """Inverse of signed log: sign(x) * (exp(|x|) - 1)"""
    return np.sign(x) * (np.exp(np.abs(x)) - 1)

# Criar PDF para salvar todos os plots
pdf_filename = '../../Data/Data_Exploration/linear_or_log_scatter_plot.pdf'
with PdfPages(pdf_filename) as pdf:

    # Loop por todas as variáveis
    for var in vars_to_plot:
        print(f"Plotando {var}...")

        # Criar figura com 4 subplots
        fig, axes = plt.subplots(2, 2, figsize=(20, 16))
        fig.suptitle(f'{var} - Comparação de Transformações', fontsize=16, y=0.95)

        # Configurações dos 4 tipos de plot
        plot_configs = [
            ('ros_p vs ' + var, 'ros_p', var, False, False),           # ros_p vs var
            ('ros_p vs log(' + var + ')', 'ros_p', var, False, True),  # ros_p vs log(var)
            ('log(ros_p) vs ' + var, 'ros_p', var, True, False),       # log(ros_p) vs var
            ('log(ros_p) vs log(' + var + ')', 'ros_p', var, True, True)  # log(ros_p) vs log(var)
        ]

        for idx, (title, y_col, x_col, log_y, log_x) in enumerate(plot_configs):
            ax = axes[idx // 2, idx % 2]
            
            # Preparar dados
            x_original = pd.to_numeric(df_model[x_col], errors='coerce')
            y_original = pd.to_numeric(df_model[y_col], errors='coerce')
            
            # Aplicar transformações
            if log_x:
                x = signed_log(x_original)  # signed log
                x_linear = x_original
            else:
                x = x_original
                x_linear = x_original
                
            if log_y:
                y = signed_log(y_original)  # signed log
                y_linear = y_original
            else:
                y = y_original
                y_linear = y_original
            
            # Filtrar dados válidos
            mask = x.notna() & y.notna()
            x_clean = x[mask]
            y_clean = y[mask]
            x_linear_clean = x_linear[mask]
            y_linear_clean = y_linear[mask]
            
            if len(x_clean) < 5:
                ax.text(0.5, 0.5, 'Dados insuficientes', ha='center', va='center', transform=ax.transAxes)
                ax.set_title(title)
                continue

            try:
                # Regressão linear
                coeffs = np.polyfit(x_clean, y_clean, 1)
                y_pred = np.polyval(coeffs, x_clean)
                r2 = r2_score(y_clean, y_pred)
                
                # Correlação de Pearson
                r, p_value = stats.pearsonr(x_clean, y_clean)
                
                # Scatter plot com densidade
                xy = np.vstack([x_clean, y_clean])
                z = gaussian_kde(xy)(xy)
                idx = z.argsort()
                x_sorted, y_sorted, z_sorted = x_clean.iloc[idx], y_clean.iloc[idx], z[idx]
                
                scatter = ax.scatter(x_sorted, y_sorted, c=z_sorted, s=20, cmap='viridis', alpha=0.6)
                
                # Linha de regressão
                x_fit = np.linspace(x_clean.min(), x_clean.max(), 100)
                y_fit = np.polyval(coeffs, x_fit)
                ax.plot(x_fit, y_fit, 'r--', linewidth=2, 
                        label=f'r={r:.3f}, R²={r2:.3f}\np={p_value:.2e}')
                
                # Configurar eixos com escalas duplas para transformações log
                if log_x:
                    # Criar eixo secundário para x em escala linear
                    ax2_x = ax.twiny()
                    
                    # Definir limites baseados nos dados
                    x_log_min, x_log_max = x_clean.min(), x_clean.max()
                    ax.set_xlim(x_log_min, x_log_max)
                    
                    # Escolher ticks "normais" para o eixo log (valores redondos)
                    x_log_range = x_log_max - x_log_min
                    
                    # Escolher step baseado no range (mais ticks para eixos secundários)
                    if x_log_range <= 1:
                        step_primary = 0.2
                        step_secondary = 0.1  # Mais ticks no secundário
                    elif x_log_range <= 3:
                        step_primary = 0.5
                        step_secondary = 0.2  # Mais ticks no secundário
                    else:
                        step_primary = 1.0
                        step_secondary = 0.5  # Mais ticks no secundário
                    
                    # Criar ticks "normais" para eixo principal
                    x_log_ticks_primary = np.arange(
                        np.floor(x_log_min / step_primary) * step_primary,
                        np.ceil(x_log_max / step_primary) * step_primary + step_primary/2,
                        step_primary
                    )
                    
                    # Criar MAIS ticks para eixo secundário
                    x_log_ticks_secondary = np.arange(
                        np.floor(x_log_min / step_secondary) * step_secondary,
                        np.ceil(x_log_max / step_secondary) * step_secondary + step_secondary/2,
                        step_secondary
                    )
                    
                    # Filtrar ticks dentro dos limites
                    x_log_ticks_primary = [tick for tick in x_log_ticks_primary if x_log_min <= tick <= x_log_max]
                    x_log_ticks_secondary = [tick for tick in x_log_ticks_secondary if x_log_min <= tick <= x_log_max]
                    
                    # Se não há ticks suficientes, usar divisão linear
                    if len(x_log_ticks_primary) < 3:
                        x_log_ticks_primary = np.linspace(x_log_min, x_log_max, 5)
                        x_log_ticks_secondary = np.linspace(x_log_min, x_log_max, 9)  # Mais ticks
                    
                    # Converter ticks para linear
                    x_linear_ticks_primary = [signed_exp(tick) for tick in x_log_ticks_primary]
                    x_linear_ticks_secondary = [signed_exp(tick) for tick in x_log_ticks_secondary]
                    
                    # Aplicar os ticks nos eixos
                    ax.set_xticks(x_log_ticks_primary)
                    ax.set_xticklabels([f'{tick:.1f}' for tick in x_log_ticks_primary])
                    
                    # Configurar eixo linear secundário com MAIS TICKS
                    x_linear_min = signed_exp(x_log_min)
                    x_linear_max = signed_exp(x_log_max)
                    ax2_x.set_xlim(x_linear_min, x_linear_max)
                    ax2_x.set_xticks(x_linear_ticks_secondary)  # Usar os ticks extras
                    ax2_x.set_xticklabels([f'{tick:.0f}' if tick >= 10 else f'{tick:.1f}' for tick in x_linear_ticks_secondary])
                    ax2_x.set_xlabel(f'{var} (linear)')
                    ax2_x.tick_params(axis='x', labelsize=7)  # Texto menor para mais ticks
                
                if log_y:
                    # Criar eixo secundário para y em escala linear
                    ax2_y = ax.twinx()
                    
                    # Definir limites baseados nos dados
                    y_log_min, y_log_max = y_clean.min(), y_clean.max()
                    ax.set_ylim(y_log_min, y_log_max)
                    
                    # Escolher ticks "normais" para o eixo log (valores redondos)
                    y_log_range = y_log_max - y_log_min
                    
                    # Escolher step baseado no range (mais ticks para eixos secundários)
                    if y_log_range <= 1:
                        step_primary = 0.2
                        step_secondary = 0.1  # Mais ticks no secundário
                    elif y_log_range <= 3:
                        step_primary = 0.5
                        step_secondary = 0.2  # Mais ticks no secundário
                    else:
                        step_primary = 1.0
                        step_secondary = 0.5  # Mais ticks no secundário
                    
                    # Criar ticks "normais" para eixo principal
                    y_log_ticks_primary = np.arange(
                        np.floor(y_log_min / step_primary) * step_primary,
                        np.ceil(y_log_max / step_primary) * step_primary + step_primary/2,
                        step_primary
                    )
                    
                    # Criar MAIS ticks para eixo secundário
                    y_log_ticks_secondary = np.arange(
                        np.floor(y_log_min / step_secondary) * step_secondary,
                        np.ceil(y_log_max / step_secondary) * step_secondary + step_secondary/2,
                        step_secondary
                    )
                    
                    # Filtrar ticks dentro dos limites
                    y_log_ticks_primary = [tick for tick in y_log_ticks_primary if y_log_min <= tick <= y_log_max]
                    y_log_ticks_secondary = [tick for tick in y_log_ticks_secondary if y_log_min <= tick <= y_log_max]
                    
                    # Se não há ticks suficientes, usar divisão linear
                    if len(y_log_ticks_primary) < 3:
                        y_log_ticks_primary = np.linspace(y_log_min, y_log_max, 5)
                        y_log_ticks_secondary = np.linspace(y_log_min, y_log_max, 9)  # Mais ticks
                    
                    # Converter ticks para linear
                    y_linear_ticks_primary = [signed_exp(tick) for tick in y_log_ticks_primary]
                    y_linear_ticks_secondary = [signed_exp(tick) for tick in y_log_ticks_secondary]
                    
                    # Aplicar os ticks nos eixos
                    ax.set_yticks(y_log_ticks_primary)
                    ax.set_yticklabels([f'{tick:.1f}' for tick in y_log_ticks_primary])
                    
                    # Configurar eixo linear secundário com MAIS TICKS
                    y_linear_min = signed_exp(y_log_min)
                    y_linear_max = signed_exp(y_log_max)
                    ax2_y.set_ylim(y_linear_min, y_linear_max)
                    ax2_y.set_yticks(y_linear_ticks_secondary)  # Usar os ticks extras
                    ax2_y.set_yticklabels([f'{tick:.0f}' if tick >= 10 else f'{tick:.1f}' for tick in y_linear_ticks_secondary])
                    ax2_y.set_ylabel('ros_p (linear)')
                    ax2_y.tick_params(axis='y', labelsize=7)  # Texto menor para mais ticks
                
                # Grid apenas nos ticks do eixo log principal
                ax.grid(True, alpha=0.3)
                
                ax.set_title(title)
                ax.legend()
                
                # Labels dos eixos principais
                x_label = f'log({var})' if log_x else var
                y_label = 'log(ros_p)' if log_y else 'ros_p'
                ax.set_xlabel(x_label)
                ax.set_ylabel(y_label)
                
            except Exception as e:
                ax.text(0.5, 0.5, f'Erro: {str(e)}', ha='center', va='center', transform=ax.transAxes)
                ax.set_title(title)

        plt.tight_layout(pad=3)
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)  # Fechar figura para não mostrar

print(f"Todos os plots salvos em '{pdf_filename}'")

Plotando inidoy...


Plotando enddoy...
Plotando duration_p...
Plotando elev_av...
Plotando aspect_av...
Plotando landform...
Plotando land_use...
Plotando 1_3y_fir_p...
Plotando 3_8y_fir_p...
Plotando 8_ny_fir_p...
Plotando fuel_model...
Plotando f_load_av...
Plotando sW_1m_av...
Plotando sW_3m_av...
Plotando sW_7_av...
Plotando sW_28_av...
Plotando sW_100_av...
Plotando sW_289_av...
Plotando t_2m_C_av...
Plotando d_2m_C_av...
Plotando rh_2m_av...
Plotando VPD_Pa_av...
Plotando sP_hPa_av...
Plotando gp_m2s2_av...
Plotando dfmc_av...
Plotando HDW_av...
Plotando Haines_av...
Plotando FWI_12h_av...
Plotando DC_12h_av...
Plotando FFMC_12h_a...
Plotando wv10_kh_av...
Plotando wdir10_av...
Plotando wv100_k_av...
Plotando wdir100_av...
Plotando Recirc...
Plotando CircVar...
Plotando t_950_av...
Plotando t_850_av...
Plotando t_700_av...
Plotando t_500_av...
Plotando t_300_av...
Plotando rh_950_av...
Plotando rh_850_av...
Plotando rh_700_av...
Plotando rh_500_av...
Plotando rh_300_av...
Plotando wv_950_av...
Plota

In [6]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import gaussian_kde, linregress
from sklearn.metrics import r2_score

def signed_log(x):
    """Apply signed log transformation: sign(x) * log(|x| + 1)"""
    return np.sign(x) * np.log(np.abs(x) + 1)

def signed_exp(x):
    """Inverse of signed log: sign(x) * (exp(|x|) - 1)"""
    return np.sign(x) * (np.exp(np.abs(x)) - 1)

# Criar PDF para salvar todos os plots
pdf_filename = '../../Data/Data_Exploration/linear_or_log_kde_density_plots.pdf'
with PdfPages(pdf_filename) as pdf:
    
    for var in vars_to_plot:
        print(f"Plotando KDE para {var}...")

        # Criar figura com 4 subplots
        fig, axes = plt.subplots(2, 2, figsize=(22, 16))  # Aumentei a largura para dar mais espaço
        fig.suptitle(f'{var} - KDE Density Plots', fontsize=16, y=0.95)

        # Configurações dos 4 tipos de plot
        plot_configs = [
            ('ros_p vs ' + var, 'ros_p', var, False, False),           # ros_p vs var
            ('ros_p vs log(' + var + ')', 'ros_p', var, False, True),  # ros_p vs log(var)
            ('log(ros_p) vs ' + var, 'ros_p', var, True, False),       # log(ros_p) vs var
            ('log(ros_p) vs log(' + var + ')', 'ros_p', var, True, True)  # log(ros_p) vs log(var)
        ]

        for idx, (title, y_col, x_col, log_y, log_x) in enumerate(plot_configs):
            ax = axes[idx // 2, idx % 2]
            
            # Preparar dados
            x_original = pd.to_numeric(df_model[x_col], errors='coerce')
            y_original = pd.to_numeric(df_model[y_col], errors='coerce')
            
            # Aplicar transformações
            if log_x:
                x = signed_log(x_original)  # signed log
                x_linear = x_original
            else:
                x = x_original
                x_linear = x_original
                
            if log_y:
                y = signed_log(y_original)  # signed log
                y_linear = y_original
            else:
                y = y_original
                y_linear = y_original
            
            # Filtrar dados válidos
            mask = x.notna() & y.notna()
            x_clean = x[mask]
            y_clean = y[mask]
            x_linear_clean = x_linear[mask]
            y_linear_clean = y_linear[mask]
            
            if len(x_clean) < 5:
                ax.text(0.5, 0.5, 'Dados insuficientes', ha='center', va='center', transform=ax.transAxes)
                ax.set_title(title)
                continue

            try:
                # Estatísticas
                slope, intercept, r_value, p_value, std_err = linregress(x_clean, y_clean)
                r2 = r_value**2

                # KDE
                xy = np.vstack([x_clean, y_clean])
                kde = gaussian_kde(xy)
                x_min, x_max = x_clean.min(), x_clean.max()
                y_min, y_max = y_clean.min(), y_clean.max()
                x_range = x_max - x_min
                y_range = y_max - y_min
                x_min -= 0.1 * x_range
                x_max += 0.1 * x_range
                y_min -= 0.1 * y_range
                y_max += 0.1 * y_range
                
                xx, yy = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
                positions = np.vstack([xx.ravel(), yy.ravel()])
                z = np.reshape(kde(positions).T, xx.shape)
                
                # Plot KDE
                contourf = ax.contourf(xx, yy, z, levels=20, cmap='viridis', alpha=0.8)
                
                # Colorbar com mais espaço
                cbar = plt.colorbar(contourf, ax=ax, label='Densidade', pad=0.08)  # Aumentei o pad
                cbar.ax.tick_params(labelsize=8)  # Texto menor na colorbar

                # Linha de regressão
                y_pred = intercept + slope * x_clean
                ax.plot(x_clean, y_pred, 'r--', linewidth=2, 
                        label=f'r={r_value:.3f}, R²={r2:.3f}\np={p_value:.2e}')

                # Configurar eixos com escalas duplas para transformações log
                if log_x:
                    # Criar eixo secundário para x em escala linear
                    ax2_x = ax.twiny()
                    
                    # Definir limites baseados nos dados
                    x_log_min, x_log_max = x_clean.min(), x_clean.max()
                    ax.set_xlim(x_log_min, x_log_max)
                    
                    # Escolher ticks "normais" para o eixo log
                    x_log_range = x_log_max - x_log_min
                    
                    # Escolher step baseado no range (mais ticks para eixos secundários)
                    if x_log_range <= 1:
                        step_primary = 0.2
                        step_secondary = 0.1
                    elif x_log_range <= 3:
                        step_primary = 0.5
                        step_secondary = 0.2
                    else:
                        step_primary = 1.0
                        step_secondary = 0.5
                    
                    # Criar ticks para eixo principal
                    x_log_ticks_primary = np.arange(
                        np.floor(x_log_min / step_primary) * step_primary,
                        np.ceil(x_log_max / step_primary) * step_primary + step_primary/2,
                        step_primary
                    )
                    
                    # Criar MAIS ticks para eixo secundário
                    x_log_ticks_secondary = np.arange(
                        np.floor(x_log_min / step_secondary) * step_secondary,
                        np.ceil(x_log_max / step_secondary) * step_secondary + step_secondary/2,
                        step_secondary
                    )
                    
                    # Filtrar ticks dentro dos limites
                    x_log_ticks_primary = [tick for tick in x_log_ticks_primary if x_log_min <= tick <= x_log_max]
                    x_log_ticks_secondary = [tick for tick in x_log_ticks_secondary if x_log_min <= tick <= x_log_max]
                    
                    # Se não há ticks suficientes, usar divisão linear
                    if len(x_log_ticks_primary) < 3:
                        x_log_ticks_primary = np.linspace(x_log_min, x_log_max, 5)
                        x_log_ticks_secondary = np.linspace(x_log_min, x_log_max, 9)
                    
                    # Converter ticks para linear
                    x_linear_ticks_primary = [signed_exp(tick) for tick in x_log_ticks_primary]
                    x_linear_ticks_secondary = [signed_exp(tick) for tick in x_log_ticks_secondary]
                    
                    # Aplicar os ticks nos eixos
                    ax.set_xticks(x_log_ticks_primary)
                    ax.set_xticklabels([f'{tick:.1f}' for tick in x_log_ticks_primary])
                    
                    # Configurar eixo linear secundário com MAIS TICKS
                    x_linear_min = signed_exp(x_log_min)
                    x_linear_max = signed_exp(x_log_max)
                    ax2_x.set_xlim(x_linear_min, x_linear_max)
                    ax2_x.set_xticks(x_linear_ticks_secondary)
                    ax2_x.set_xticklabels([f'{tick:.0f}' if tick >= 10 else f'{tick:.1f}' for tick in x_linear_ticks_secondary])
                    ax2_x.set_xlabel(f'{var} (linear)')
                    ax2_x.tick_params(axis='x', labelsize=7)
                
                if log_y:
                    # Criar eixo secundário para y em escala linear
                    ax2_y = ax.twinx()
                    
                    # Definir limites baseados nos dados
                    y_log_min, y_log_max = y_clean.min(), y_clean.max()
                    ax.set_ylim(y_log_min, y_log_max)
                    
                    # Escolher ticks "normais" para o eixo log
                    y_log_range = y_log_max - y_log_min
                    
                    # Escolher step baseado no range (mais ticks para eixos secundários)
                    if y_log_range <= 1:
                        step_primary = 0.2
                        step_secondary = 0.1
                    elif y_log_range <= 3:
                        step_primary = 0.5
                        step_secondary = 0.2
                    else:
                        step_primary = 1.0
                        step_secondary = 0.5
                    
                    # Criar ticks para eixo principal
                    y_log_ticks_primary = np.arange(
                        np.floor(y_log_min / step_primary) * step_primary,
                        np.ceil(y_log_max / step_primary) * step_primary + step_primary/2,
                        step_primary
                    )
                    
                    # Criar MAIS ticks para eixo secundário
                    y_log_ticks_secondary = np.arange(
                        np.floor(y_log_min / step_secondary) * step_secondary,
                        np.ceil(y_log_max / step_secondary) * step_secondary + step_secondary/2,
                        step_secondary
                    )
                    
                    # Filtrar ticks dentro dos limites
                    y_log_ticks_primary = [tick for tick in y_log_ticks_primary if y_log_min <= tick <= y_log_max]
                    y_log_ticks_secondary = [tick for tick in y_log_ticks_secondary if y_log_min <= tick <= y_log_max]
                    
                    # Se não há ticks suficientes, usar divisão linear
                    if len(y_log_ticks_primary) < 3:
                        y_log_ticks_primary = np.linspace(y_log_min, y_log_max, 5)
                        y_log_ticks_secondary = np.linspace(y_log_min, y_log_max, 9)
                    
                    # Converter ticks para linear
                    y_linear_ticks_primary = [signed_exp(tick) for tick in y_log_ticks_primary]
                    y_linear_ticks_secondary = [signed_exp(tick) for tick in y_log_ticks_secondary]
                    
                    # Aplicar os ticks nos eixos
                    ax.set_yticks(y_log_ticks_primary)
                    ax.set_yticklabels([f'{tick:.1f}' for tick in y_log_ticks_primary])
                    
                    # Configurar eixo linear secundário com MAIS TICKS
                    y_linear_min = signed_exp(y_log_min)
                    y_linear_max = signed_exp(y_log_max)
                    ax2_y.set_ylim(y_linear_min, y_linear_max)
                    ax2_y.set_yticks(y_linear_ticks_secondary)
                    ax2_y.set_yticklabels([f'{tick:.0f}' if tick >= 10 else f'{tick:.1f}' for tick in y_linear_ticks_secondary])
                    ax2_y.set_ylabel('ros_p (linear)')
                    ax2_y.tick_params(axis='y', labelsize=7)
                
                # Grid apenas nos ticks do eixo log principal
                ax.grid(True, alpha=0.3)
                
                ax.set_title(title)
                ax.legend()
                
                # Labels dos eixos principais
                x_label = f'log({var})' if log_x else var
                y_label = 'log(ros_p)' if log_y else 'ros_p'
                ax.set_xlabel(x_label)
                ax.set_ylabel(y_label)
                
            except Exception as e:
                ax.text(0.5, 0.5, f'Erro: {str(e)}', ha='center', va='center', transform=ax.transAxes)
                ax.set_title(title)

        plt.tight_layout(pad=4.0)  # Aumentei o padding geral
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)  # Fechar figura para não mostrar

print(f"Todos os KDE plots salvos em '{pdf_filename}'")

Plotando KDE para inidoy...
Plotando KDE para enddoy...
Plotando KDE para duration_p...
Plotando KDE para elev_av...
Plotando KDE para aspect_av...
Plotando KDE para landform...
Plotando KDE para land_use...
Plotando KDE para 1_3y_fir_p...
Plotando KDE para 3_8y_fir_p...
Plotando KDE para 8_ny_fir_p...
Plotando KDE para fuel_model...
Plotando KDE para f_load_av...
Plotando KDE para sW_1m_av...
Plotando KDE para sW_3m_av...
Plotando KDE para sW_7_av...
Plotando KDE para sW_28_av...
Plotando KDE para sW_100_av...
Plotando KDE para sW_289_av...
Plotando KDE para t_2m_C_av...
Plotando KDE para d_2m_C_av...
Plotando KDE para rh_2m_av...
Plotando KDE para VPD_Pa_av...
Plotando KDE para sP_hPa_av...
Plotando KDE para gp_m2s2_av...
Plotando KDE para dfmc_av...
Plotando KDE para HDW_av...
Plotando KDE para Haines_av...
Plotando KDE para FWI_12h_av...
Plotando KDE para DC_12h_av...
Plotando KDE para FFMC_12h_a...
Plotando KDE para wv10_kh_av...
Plotando KDE para wdir10_av...
Plotando KDE para wv